# Модель сегментации Keras

In [1]:
import numpy as np, pandas as pd, os
import cv2
import keras
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt, time
import random
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Изменяю структуру трейн датафрейма так, чтобы для каждого класса дефекта был отдельный столбец с RLE кодировкой

images = os.listdir('../input/severstal-steel-defect-detection/train_images')
train = pd.read_csv('../input/severstal-steel-defect-detection/train.csv')

tmp = pd.DataFrame()
tmp['ImageId_ClassId'] = [images[i] + '_' + str(j) for i in range(len(images)) for j in range(1, 4 + 1)]
train['ImageId_ClassId'] = train['ImageId'] + '_' + train['ClassId'].astype(str)
train = tmp.merge(train, on = ['ImageId_ClassId'], how = 'left')
train['ImageId'] = train['ImageId_ClassId'].map(lambda x: x.split('.')[0]+'.jpg')

train2 = pd.DataFrame({'ImageId':train['ImageId'][::4]})
train2['e1'] = train['EncodedPixels'][::4].values
train2['e2'] = train['EncodedPixels'][1::4].values
train2['e3'] = train['EncodedPixels'][2::4].values
train2['e4'] = train['EncodedPixels'][3::4].values
train2.reset_index(inplace = True, drop = True)
train2.fillna('',inplace=True) # заменяю NaN   

train = train2.drop_duplicates()
train.head()

,ImageId,e1,e2,e3,e4
0,74c8a2d5a.jpg,,,,
1,0d617d477.jpg,,,109771 7 110014 20 110257 33 110500 46 110743 ...,
2,66e6c8a78.jpg,,,,
3,d2670190d.jpg,,,198674 2 198928 5 199182 8 199435 12 199689 15...,
4,2e4fefc28.jpg,,,9239 1 9494 2 9749 3 10005 4 10260 5 10515 6 1...,


In [3]:
# Проверка
train.shape

(12568, 5)

In [4]:
# Проверка
train['ImageId'].value_counts()

88ab9068d.jpg    1
5c876d114.jpg    1
1ada6a7ac.jpg    1
17e6e4b30.jpg    1
1a9c38991.jpg    1
                ..
34ee04d85.jpg    1
e39c0bbac.jpg    1
25bc3a693.jpg    1
90524c39b.jpg    1
d85d973bd.jpg    1
Name: ImageId, Length: 12568, dtype: int64

In [5]:
def rle2mask(rle, imgshape):
    height = imgshape[0]
    width = imgshape[1]
    mask = np.zeros(width * height).astype(np.uint8)
    rle_array = np.asarray([int(x) for x in rle.split()]) # RLE string to array
    start_pixels = rle_array[0::2]
    run_lengths = rle_array[1::2]

    for i in range(len(start_pixels)):
        mask[start_pixels[i]:start_pixels[i] + run_lengths[i]] = 1 # одномерный массив, состоящий из 256 * 1600 = 409600 элементов
    mask = mask.reshape(width, height) # 1600 строк, 256 столбцов
    return mask.T # 256 строк, 1600 столбцов

In [6]:
# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
# Класс, который будет использоваться для подачи данных в режиме реального времени в модель Keras.

class DataGenerator(keras.utils.Sequence):
    def __init__(self, df, batch_size = 10, subset = "train", shuffle = False, preprocess = None):
        super().__init__()
        self.df = df
        self.shuffle = shuffle
        self.subset = subset
        self.batch_size = batch_size
        self.preprocess = preprocess

        
        if (self.subset == "train") or (self.subset == "val"):
            self.data_path = '../input/severstal-steel-defect-detection/train_images/'
        elif self.subset == "test":
            self.data_path = '../input/severstal-steel-defect-detection/test_images/'
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size)) # возвращает количество батчей за эпоху
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df)) # возвращает одномерный массив с равномерно разнесенными значениями внутри заданного интервала
        if self.shuffle == True:
            np.random.shuffle(self.indexes) # перемешивает входной датасет df каждую эпоху
    
    def augment(self, images, masks):
        data_gen_args = dict(horizontal_flip = True, vertical_flip = True)
        image_datagen = ImageDataGenerator(**data_gen_args)
        mask_datagen = ImageDataGenerator(**data_gen_args)
        seed = random.randint(1, 1000)
        #image_datagen.fit(images, augment=False, rounds=1, seed=seed)
        #mask_datagen.fit(masks, augment=False, rounds=1, seed=seed)
        images_aug = image_datagen.flow(images, seed=seed, batch_size = self.batch_size)[0]
        masks_aug = mask_datagen.flow(masks, seed=seed, batch_size = self.batch_size)[0]
        return images_aug, masks_aug
    
    def augment1(self, img, mask):
        rand = random.randint(1, 1000)
        if(rand > 900): return cv2.flip(img, 0), cv2.flip(mask, 0) #отражает изображение по вертикали
        if(rand < 100): return cv2.flip(img, 1), cv2.flip(mask, 1) #отражает изображение по горизонтали
        else: return img, mask
    
    def __getitem__(self, index): 
        images = np.empty((self.batch_size, 256, 1600, 3), dtype=np.float32) # массив, состоящий из batch_size изображений (256x1600x3)
        masks = np.empty((self.batch_size, 256, 1600, 4), dtype=np.int8) # массив, состоящий из batch_size изображений с масками
        indexes = self.indexes[index * self.batch_size:(index+1) * self.batch_size]
        
        for i, img_id in enumerate(self.df['ImageId'].iloc[indexes]):
            img = cv2.imread(self.data_path + img_id) # цветовым пространством по умолчанию в OpenCV является BGR
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # чтобы исправить это, используется cvtColor(image, flag) и рассмотренный выше флаг
            images[i,] = img.astype(np.float32) / 255.
            if (self.subset == "train") or (self.subset == "val"): 
                for j in range(4):
                    masks[i,:,:,j] = rle2mask(self.df['e' + str(j+1)].iloc[indexes[i]], img.shape) # 4 канала, нулевой канал - дефект первого типа, третий канал - дефект четвертого типа
            if self.subset == "train":
                images[i,], masks[i,] = self.augment1(images[i,], masks[i,])
        if self.subset == 'train':
            #images, masks = self.augment(images, masks)
            return images, masks
        if self.subset == 'val':
            return images, masks
        else: return images # если test

# Модель
В качестве модели использовалась архитекстура U-Net с энкодером ResNet34, веса которого были предобучены на ImageNet-e, взято с https://github.com/qubvel/segmentation_models Документация https://segmentation-models.readthedocs.io/en/latest/tutorial.html Модель решала задачу многоклассовой сегментации, то есть одновременно определяла – есть ли на изображении дефект и если есть, то выводила его расположение и класс. Модель тренировалась на полных изображениях 1600x256, без изменения размера. Обучение длилось 60 эпох на видеокарте Tesla P100, это заняло около 12 часов. Первые 40 эпох скорость обучения (learning rate) была равна 1e-3, последние 20 эпох скорость обучена была снижена до 1e-4.


In [7]:
! pip install segmentation-models

     |████████████████████████████████| 50 kB 3.5 MB/s  eta 0:00:01


In [8]:
# https://github.com/CyberZHG/keras-radam/blob/master/keras_radam/optimizer_v2.py
# Дайс метрика качества

def dice_coef(y_true, y_pred, smooth = 1, THRESHOLD = 0.45):
    tf.to_float = lambda x: tf.cast(x, tf.float32)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    y_pred_f = tf.to_float(y_pred_f > THRESHOLD)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth) # smooth необходимо, чтобы не возникало деление на ноль

In [9]:
def jaccard_distance_loss(y_true, y_pred, smooth = 1):
    """
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    
    The jaccard distance loss is usefull for unbalanced datasets. This has been
    shifted so it converges on 0 and is smoothed to avoid exploding or disapearing
    gradient.
    
    Ref: https://en.wikipedia.org/wiki/Jaccard_index
    
    @url: https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
    @author: wassname
    """
    #tf.to_float = lambda x: tf.cast(x, tf.float32)
    #y_pred = tf.to_float(y_pred > 0.45)
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

In [17]:
def Dice_Coef(y_true, y_pred, smooth = 1):
    
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    intersection = K.sum(y_true_f * y_pred_f)
    
    return (2*intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def Dice_Loss(y_true, y_pred):
    return 1.0 - Dice_Coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    return keras.losses.binary_crossentropy(y_true, y_pred) + Dice_Loss(y_true, y_pred)

def wbce_dice_loss(y_true, y_pred):
    return weighted_bce()(y_true, y_pred) + Dice_Loss(y_true, y_pred)

def weighted_bce(weight = 0.6):
    
    def convert_2_logits(y_pred):
        y_pred = tf.clip_by_value(y_pred, K.epsilon(), 1 - K.epsilon())
        return tf.math.log(y_pred / (1-y_pred))
    
    def weighted_binary_crossentropy(y_true, y_pred):
        y_pred = convert_2_logits(y_pred)
        loss = tf.nn.weighted_cross_entropy_with_logits(logits = y_pred, labels = y_true, pos_weight = weight)
        return loss
    
    return weighted_binary_crossentropy

In [11]:
# Взято с https://github.com/CyberZHG/keras-radam/blob/master/keras_radam/optimizer_v2.py

from tensorflow.python.keras.optimizer_v2.optimizer_v2 import OptimizerV2
from tensorflow.python import ops, math_ops, state_ops, control_flow_ops
from tensorflow.python.keras import backend as K

__all__ = ['RAdam']


class RAdam(OptimizerV2):
    """RAdam optimizer.
    According to the paper
    [On The Variance Of The Adaptive Learning Rate And Beyond](https://arxiv.org/pdf/1908.03265v1.pdf).
    """

    def __init__(self,
                 learning_rate=0.001,
                 beta_1=0.9,
                 beta_2=0.999,
                 epsilon=1e-7,
                 weight_decay=0.,
                 amsgrad=False,
                 total_steps=0,
                 warmup_proportion=0.1,
                 min_lr=0.,
                 name='RAdam',
                 **kwargs):
        r"""Construct a new Adam optimizer.
        Args:
            learning_rate: A Tensor or a floating point value.    The learning rate.
            beta_1: A float value or a constant float tensor. The exponential decay
                rate for the 1st moment estimates.
            beta_2: A float value or a constant float tensor. The exponential decay
                rate for the 2nd moment estimates.
            epsilon: A small constant for numerical stability. This epsilon is
                "epsilon hat" in the Kingma and Ba paper (in the formula just before
                Section 2.1), not the epsilon in Algorithm 1 of the paper.
            weight_decay: A floating point value. Weight decay for each param.
            amsgrad: boolean. Whether to apply AMSGrad variant of this algorithm from
                the paper "On the Convergence of Adam and beyond".
            total_steps: An integer. Total number of training steps.
                Enable warmup by setting a positive value.
            warmup_proportion: A floating point value. The proportion of increasing steps.
            min_lr: A floating point value. Minimum learning rate after warmup.
            name: Optional name for the operations created when applying gradients.
                Defaults to "Adam".    @compatibility(eager) When eager execution is
                enabled, `learning_rate`, `beta_1`, `beta_2`, and `epsilon` can each be
                a callable that takes no arguments and returns the actual value to use.
                This can be useful for changing these values across different
                invocations of optimizer functions. @end_compatibility
            **kwargs: keyword arguments. Allowed to be {`clipnorm`, `clipvalue`, `lr`,
                `decay`}. `clipnorm` is clip gradients by norm; `clipvalue` is clip
                gradients by value, `decay` is included for backward compatibility to
                allow time inverse decay of learning rate. `lr` is included for backward
                compatibility, recommended to use `learning_rate` instead.
        """

        super(RAdam, self).__init__(name, **kwargs)
        self._set_hyper('learning_rate', kwargs.get('lr', learning_rate))
        self._set_hyper('beta_1', beta_1)
        self._set_hyper('beta_2', beta_2)
        self._set_hyper('decay', self._initial_decay)
        self._set_hyper('weight_decay', weight_decay)
        self._set_hyper('total_steps', float(total_steps))
        self._set_hyper('warmup_proportion', warmup_proportion)
        self._set_hyper('min_lr', min_lr)
        self.epsilon = epsilon or K.epsilon()
        self.amsgrad = amsgrad
        self._initial_weight_decay = weight_decay
        self._initial_total_steps = total_steps

    def _create_slots(self, var_list):
        for var in var_list:
            self.add_slot(var, 'm')
        for var in var_list:
            self.add_slot(var, 'v')
        if self.amsgrad:
            for var in var_list:
                self.add_slot(var, 'vhat')

    def set_weights(self, weights):
        params = self.weights
        num_vars = int((len(params) - 1) / 2)
        if len(weights) == 3 * num_vars + 1:
            weights = weights[:len(params)]
        super(RAdam, self).set_weights(weights)

    def _resource_apply_dense(self, grad, var):
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype)
        m = self.get_slot(var, 'm')
        v = self.get_slot(var, 'v')
        beta_1_t = self._get_hyper('beta_1', var_dtype)
        beta_2_t = self._get_hyper('beta_2', var_dtype)
        epsilon_t = ops.convert_to_tensor(self.epsilon, var_dtype)
        local_step = math_ops.cast(self.iterations + 1, var_dtype)
        beta_1_power = math_ops.pow(beta_1_t, local_step)
        beta_2_power = math_ops.pow(beta_2_t, local_step)

        if self._initial_total_steps > 0:
            total_steps = self._get_hyper('total_steps', var_dtype)
            warmup_steps = total_steps * self._get_hyper('warmup_proportion', var_dtype)
            min_lr = self._get_hyper('min_lr', var_dtype)
            decay_steps = K.maximum(total_steps - warmup_steps, 1)
            decay_rate = (min_lr - lr_t) / decay_steps
            lr_t = tf.where(
                local_step <= warmup_steps,
                lr_t * (local_step / warmup_steps),
                lr_t + decay_rate * K.minimum(local_step - warmup_steps, decay_steps),
            )

        sma_inf = 2.0 / (1.0 - beta_2_t) - 1.0
        sma_t = sma_inf - 2.0 * local_step * beta_2_power / (1.0 - beta_2_power)

        m_t = state_ops.assign(m,
                               beta_1_t * m + (1.0 - beta_1_t) * grad,
                               use_locking=self._use_locking)
        m_corr_t = m_t / (1.0 - beta_1_power)

        v_t = state_ops.assign(v,
                               beta_2_t * v + (1.0 - beta_2_t) * math_ops.square(grad),
                               use_locking=self._use_locking)
        if self.amsgrad:
            vhat = self.get_slot(var, 'vhat')
            vhat_t = state_ops.assign(vhat,
                                      math_ops.maximum(vhat, v_t),
                                      use_locking=self._use_locking)
            v_corr_t = math_ops.sqrt(vhat_t / (1.0 - beta_2_power))
        else:
            vhat_t = None
            v_corr_t = math_ops.sqrt(v_t / (1.0 - beta_2_power))

        r_t = math_ops.sqrt((sma_t - 4.0) / (sma_inf - 4.0) *
                            (sma_t - 2.0) / (sma_inf - 2.0) *
                            sma_inf / sma_t)

        var_t = tf.where(sma_t >= 5.0, r_t * m_corr_t / (v_corr_t + epsilon_t), m_corr_t)

        if self._initial_weight_decay > 0.0:
            var_t += self._get_hyper('weight_decay', var_dtype) * var

        var_update = state_ops.assign_sub(var,
                                          lr_t * var_t,
                                          use_locking=self._use_locking)

        updates = [var_update, m_t, v_t]
        if self.amsgrad:
            updates.append(vhat_t)
        return control_flow_ops.group(*updates)

    def _resource_apply_sparse(self, grad, var, indices):
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype)
        beta_1_t = self._get_hyper('beta_1', var_dtype)
        beta_2_t = self._get_hyper('beta_2', var_dtype)
        epsilon_t = ops.convert_to_tensor(self.epsilon, var_dtype)
        local_step = math_ops.cast(self.iterations + 1, var_dtype)
        beta_1_power = math_ops.pow(beta_1_t, local_step)
        beta_2_power = math_ops.pow(beta_2_t, local_step)

        if self._initial_total_steps > 0:
            total_steps = self._get_hyper('total_steps', var_dtype)
            warmup_steps = total_steps * self._get_hyper('warmup_proportion', var_dtype)
            min_lr = self._get_hyper('min_lr', var_dtype)
            decay_steps = K.maximum(total_steps - warmup_steps, 1)
            decay_rate = (min_lr - lr_t) / decay_steps
            lr_t = tf.where(
                local_step <= warmup_steps,
                lr_t * (local_step / warmup_steps),
                lr_t + decay_rate * K.minimum(local_step - warmup_steps, decay_steps),
            )

        sma_inf = 2.0 / (1.0 - beta_2_t) - 1.0
        sma_t = sma_inf - 2.0 * local_step * beta_2_power / (1.0 - beta_2_power)

        m = self.get_slot(var, 'm')
        m_scaled_g_values = grad * (1 - beta_1_t)
        m_t = state_ops.assign(m, m * beta_1_t, use_locking=self._use_locking)
        with ops.control_dependencies([m_t]):
            m_t = self._resource_scatter_add(m, indices, m_scaled_g_values)
        m_corr_t = m_t / (1.0 - beta_1_power)

        v = self.get_slot(var, 'v')
        v_scaled_g_values = (grad * grad) * (1 - beta_2_t)
        v_t = state_ops.assign(v, v * beta_2_t, use_locking=self._use_locking)
        with ops.control_dependencies([v_t]):
            v_t = self._resource_scatter_add(v, indices, v_scaled_g_values)

        if self.amsgrad:
            vhat = self.get_slot(var, 'vhat')
            vhat_t = state_ops.assign(vhat,
                                      math_ops.maximum(vhat, v_t),
                                      use_locking=self._use_locking)
            v_corr_t = math_ops.sqrt(vhat_t / (1.0 - beta_2_power))
        else:
            vhat_t = None
            v_corr_t = math_ops.sqrt(v_t / (1.0 - beta_2_power))

        r_t = math_ops.sqrt((sma_t - 4.0) / (sma_inf - 4.0) *
                            (sma_t - 2.0) / (sma_inf - 2.0) *
                            sma_inf / sma_t)

        var_t = tf.where(sma_t >= 5.0, r_t * m_corr_t / (v_corr_t + epsilon_t), m_corr_t)

        if self._initial_weight_decay > 0.0:
            var_t += self._get_hyper('weight_decay', var_dtype) * var

        var_update = self._resource_scatter_add(var, indices, tf.gather(-lr_t * var_t, indices))

        updates = [var_update, m_t, v_t]
        if self.amsgrad:
            updates.append(vhat_t)
        return control_flow_ops.group(*updates)

    def get_config(self):
        config = super(RAdam, self).get_config()
        config.update({
            'learning_rate': self._serialize_hyperparameter('learning_rate'),
            'beta_1': self._serialize_hyperparameter('beta_1'),
            'beta_2': self._serialize_hyperparameter('beta_2'),
            'decay': self._serialize_hyperparameter('decay'),
            'weight_decay': self._serialize_hyperparameter('weight_decay'),
            'epsilon': self.epsilon,
            'amsgrad': self.amsgrad,
            'total_steps': self._serialize_hyperparameter('total_steps'),
            'warmup_proportion': self._serialize_hyperparameter('warmup_proportion'),
            'min_lr': self._serialize_hyperparameter('min_lr'),
        })
        return config

In [12]:
checkpoint = ModelCheckpoint("256_resnet34.h5", monitor = 'val_dice_coef',
                             verbose = 1,save_best_only = True, mode = 'max') # сохраняет лучшую модель по Дайс метрике на валидации
callbacks_list = [checkpoint]

In [13]:
use_load_model = True

In [ ]:
if use_load_model == True:
    model = load_model('../input/pre-model8/256_resnet34 0.9018.h5', custom_objects = {'RAdam': RAdam, 'dice_coef': dice_coef, 'jaccard_distance_loss': jaccard_distance_loss})
    opt = RAdam(lr = 1e-4, min_lr = 1e-7)
    model.compile(optimizer = opt, loss = wbce_dice_loss, metrics = [dice_coef])

    train_ids, val_ids = train_test_split(range(len(train)), test_size = 0.2, random_state = 42) # разделяю выборку на трейн и валидацию
    train_batches = DataGenerator(train.iloc[train_ids], shuffle = True)
    valid_batches = DataGenerator(train.iloc[val_ids], subset = 'val')

    history = model.fit_generator(train_batches, validation_data = valid_batches, epochs = 45, verbose = 1, callbacks = callbacks_list)

Epoch 1/45
1005/1005 [==============================] - ETA: 0s - loss: 0.1432 - dice_coef: 0.8703
Epoch 00001: val_dice_coef improved from -inf to 0.90799, saving model to 256_resnet34.h5
1005/1005 [==============================] - 683s 680ms/step - loss: 0.1432 - dice_coef: 0.8703 - val_loss: 0.0997 - val_dice_coef: 0.9080
Epoch 2/45
1005/1005 [==============================] - ETA: 0s - loss: 0.1371 - dice_coef: 0.8733
Epoch 00002: val_dice_coef did not improve from 0.90799
1005/1005 [==============================] - 672s 669ms/step - loss: 0.1371 - dice_coef: 0.8733 - val_loss: 0.0989 - val_dice_coef: 0.9079
Epoch 3/45
1005/1005 [==============================] - ETA: 0s - loss: 0.1353 - dice_coef: 0.8737
Epoch 00003: val_dice_coef did not improve from 0.90799
1005/1005 [==============================] - 671s 668ms/step - loss: 0.1353 - dice_coef: 0.8737 - val_loss: 0.0997 - val_dice_coef: 0.9071
Epoch 4/45
1005/1005 [==============================] - ETA: 0s - loss: 0.1336 - dic

In [ ]:
#import segmentation_models as sm
#from segmentation_models import Unet

if use_load_model == False:
    opt = RAdam(lr = 1e-3, min_lr = 1e-7)
    model = Unet('resnet34', input_shape = (256, 1600, 3), classes = 4, activation = 'sigmoid')
    model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = [dice_coef])

    train_ids, val_ids = train_test_split(range(len(train)), test_size = 0.2, random_state = 42) # разделяю выборку на трейн и валидацию
    train_batches = DataGenerator(train.iloc[train_ids], shuffle = True)
    valid_batches = DataGenerator(train.iloc[val_ids], subset = 'val')

    history = model.fit_generator(train_batches, validation_data = valid_batches, epochs = 10, verbose = 1, callbacks = callbacks_list)

In [ ]:
batch_size = 10
ran_ids = random.choices(val_ids, k = batch_size)
val_set = train.iloc[ran_ids] # 10 случайных картинок из валидации

valid_batches = DataGenerator(val_set, subset = 'val')
preds = model.predict_generator(valid_batches, verbose = 1)

In [ ]:
# Графическое сравнение предсказаний модели с метками из валидационной выборки

THRESHOLD = 0.45
colors = [(249, 192, 12), (0, 185, 241), (114, 0, 218), (249,50,12)]

for i, batch in enumerate(valid_batches):
    for k in range(batch_size):
        fig, ax = plt.subplots(1, 2, figsize=(23, 23))
        img = batch[0][k,]
        img = np.float32(img) * 255.
        img = (img).astype(np.uint8)
        img1 = img.copy()
        has_defect = False
        extra = ' имеет дефект'
        extra_pred = ' наличие дефекта'
        for j in range(4):
            mask = batch[1][k,:,:,j].astype(np.uint8)
            if np.sum(mask) != 0: 
                has_defect = True
                extra += ' ' + str(j+1)
            contours, _ = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
            cv2.drawContours(img, contours, -1, colors[j], 2, cv2.LINE_AA)
        ax[0].imshow(img)
        if(has_defect == False): extra = '  не имеет дефектов '
        ax[0].set_title('Валидация ' + str(train['ImageId'].iloc[ran_ids[k]]) + extra, fontsize=18)
        for j in range(4):
            mask = preds[k,:,:,j]
            mask[mask >= THRESHOLD] = 1
            mask[mask < THRESHOLD] = 0
            mask = mask.astype(np.uint8)
            if np.sum(mask) != 0: 
                extra_pred += ' ' + str(j+1)
            contours, _ = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
            cv2.drawContours(img1, contours, -1, colors[j], 2, cv2.LINE_AA)
        ax[1].imshow(img1)
        if(has_defect == False): extra_pred = ' отсутствие дефектов '
        ax[1].set_title('Модель предсказывает' + extra_pred, fontsize=18)
        fig.subplots_adjust(wspace = 0.05, hspace = 0.01)

In [ ]:
# График коэффициента Дайса на трейне и тесте по эпохам
plt.figure(figsize=(15,5))
plt.plot(range(history.epoch[-1]+1),history.history['val_dice_coef'],label='val_dice_coef')
plt.plot(range(history.epoch[-1]+1),history.history['dice_coef'],label='trn_dice_coef')
plt.title('Training Accuracy'); plt.xlabel('Epoch'); plt.ylabel('Dice_coef');plt.legend(); 
plt.show()